# CLV Model Comparison

This notebook compares different approaches to Customer Lifetime Value (CLV) modeling:

1. **Historical Average**: Simple baseline using past customer behavior
2. **BG/NBD + Gamma-Gamma (MAP)**: Fast probabilistic model
3. **BG/NBD + Gamma-Gamma (MCMC)**: More accurate but slower Bayesian approach
4. **Synthetic Scenarios**: Testing models under different business conditions

## Learning Objectives

- Understand tradeoffs between model complexity and accuracy
- Compare prediction quality across different approaches
- Learn when to use MAP vs MCMC
- Validate models using synthetic data with known parameters

## Prerequisites

```bash
pip install -e .
```

## ⚠️ Data Privacy Notice

This notebook uses synthetic data. When using production customer data, ensure GDPR/CCPA compliance, anonymize identifiers, and never commit real data to version control.

## Setup: Generate Test Data

We'll use synthetic data so we can evaluate model performance objectively.

In [ ]:
from datetime import date, datetime
from dataclasses import asdict
import pandas as pd
import matplotlib.pyplot as plt
import time

from customer_base_audit.synthetic import (
    generate_customers,
    generate_transactions,
    BASELINE_SCENARIO,
    STABLE_BUSINESS_SCENARIO,
    HIGH_CHURN_SCENARIO,
)
from customer_base_audit.models.model_prep import prepare_clv_model_inputs

# Generate baseline scenario data
print("Generating synthetic data...")
customers = generate_customers(1000, date(2024, 1, 1), date(2024, 12, 31), seed=42)
transactions = generate_transactions(
    customers, date(2024, 1, 1), date(2024, 12, 31), scenario=BASELINE_SCENARIO
)

print(f"Generated {len(customers):,} customers")
print(f"Generated {len(transactions):,} transactions")

# Prepare model input data
model_data = prepare_clv_model_inputs(
    transactions=[asdict(t) for t in transactions],
    observation_start=datetime(2024, 1, 1),
    observation_end=datetime(2024, 12, 31, 23, 59, 59),
    customer_id_field="customer_id",
    timestamp_field="event_ts",
    monetary_field="unit_price",
)

print(f"\nModel-ready customers: {len(model_data)}")
print(f"Customers with 2+ purchases: {len(model_data[model_data['frequency'] >= 2])}")

## Baseline: Historical Average Method

The simplest approach: assume future behavior matches historical average.

In [ ]:
def historical_average_clv(model_data, time_periods_days=90):
    """Calculate CLV using simple historical averages."""
    # Calculate historical rates
    avg_orders_per_day = model_data["frequency"].mean() / model_data["T"].mean()
    avg_monetary = model_data["monetary_value"].mean()

    # Project forward
    predicted_purchases = avg_orders_per_day * time_periods_days
    predicted_clv = predicted_purchases * avg_monetary

    # Assign same prediction to all customers (naive approach)
    predictions = pd.DataFrame(
        {
            "customer_id": model_data["customer_id"],
            "predicted_purchases": predicted_purchases,
            "predicted_spend": avg_monetary,
            "predicted_clv": predicted_clv,
        }
    )

    return predictions


start_time = time.time()
historical_predictions = historical_average_clv(model_data, time_periods_days=90)
historical_time = time.time() - start_time

print("=== Historical Average Method ===")
print(f"Training time: {historical_time:.3f} seconds")
print(
    f"Avg predicted 90-day CLV: ${historical_predictions['predicted_clv'].mean():.2f}"
)
print(f"Total predicted revenue: ${historical_predictions['predicted_clv'].sum():,.2f}")
print("\nNote: All customers get the same prediction (no personalization)")

## Model 1: BG/NBD + Gamma-Gamma (MAP)

Maximum A Posteriori estimation - fast and effective for most use cases.

In [ ]:
from customer_base_audit.models.bg_nbd import BGNBDModelWrapper, BGNBDConfig
from customer_base_audit.models.gamma_gamma import (
    GammaGammaModelWrapper,
    GammaGammaConfig,
)

print("Training BG/NBD (MAP) model...")
start_time = time.time()

# Train BG/NBD for purchase frequency
bg_nbd_map_config = BGNBDConfig(method="map")
bg_nbd_map = BGNBDModelWrapper(bg_nbd_map_config)
bg_nbd_map.fit(model_data)

map_training_time = time.time() - start_time
print(f"✓ BG/NBD trained in {map_training_time:.2f} seconds")

# Predict purchases
map_purchase_predictions = bg_nbd_map.predict_purchases(model_data, time_periods=90.0)

# Train Gamma-Gamma for monetary value (only customers with 2+ purchases)
gamma_data = model_data[model_data["frequency"] >= 2].copy()

print("Training Gamma-Gamma (MAP) model...")
start_time = time.time()

gg_map_config = GammaGammaConfig(method="map")
gg_map = GammaGammaModelWrapper(gg_map_config)
gg_map.fit(gamma_data)

map_training_time += time.time() - start_time
print(f"✓ Gamma-Gamma trained in {time.time() - start_time:.2f} seconds")

# Predict spend
map_spend_predictions = gg_map.predict_spend(gamma_data)

# Combine predictions
map_clv = map_purchase_predictions.merge(
    map_spend_predictions, on="customer_id", how="inner"
)
map_clv["predicted_clv"] = (
    map_clv["predicted_purchases"] * map_clv["predicted_avg_spend"]
)

print("\n=== BG/NBD + Gamma-Gamma (MAP) ===")
print(f"Total training time: {map_training_time:.2f} seconds")
print(f"Customers with predictions: {len(map_clv)}")
print(f"Avg predicted 90-day CLV: ${map_clv['predicted_clv'].mean():.2f}")
print(f"Total predicted revenue: ${map_clv['predicted_clv'].sum():,.2f}")

## Model 2: BG/NBD + Gamma-Gamma (MCMC)

Markov Chain Monte Carlo - more accurate uncertainty estimates but slower.

**Note**: MCMC can be 10-100x slower than MAP. We'll use fewer customers for this demo.

In [ ]:
# Use subset for MCMC demo (to keep training time reasonable)
mcmc_sample_data = model_data.sample(min(200, len(model_data)), random_state=42)

print(f"Training BG/NBD (MCMC) on {len(mcmc_sample_data)} customers...")
print("This will take 1-2 minutes...")

start_time = time.time()

# Train BG/NBD with MCMC
bg_nbd_mcmc_config = BGNBDConfig(
    method="mcmc",
    chains=2,  # Reduced for demo
    draws=500,  # Reduced for demo (production: 2000+)
    tune=250,  # Reduced for demo (production: 1000+)
)
bg_nbd_mcmc = BGNBDModelWrapper(bg_nbd_mcmc_config)
bg_nbd_mcmc.fit(mcmc_sample_data)

mcmc_training_time = time.time() - start_time
print(f"✓ BG/NBD (MCMC) trained in {mcmc_training_time:.2f} seconds")

# Predict
mcmc_purchase_predictions = bg_nbd_mcmc.predict_purchases(
    mcmc_sample_data, time_periods=90.0
)

# For Gamma-Gamma, we'll reuse MAP predictions since MCMC would take too long
mcmc_gamma_data = mcmc_sample_data[mcmc_sample_data["frequency"] >= 2].copy()
mcmc_spend_predictions = gg_map.predict_spend(mcmc_gamma_data)

# Combine
mcmc_clv = mcmc_purchase_predictions.merge(
    mcmc_spend_predictions, on="customer_id", how="inner"
)
mcmc_clv["predicted_clv"] = (
    mcmc_clv["predicted_purchases"] * mcmc_clv["predicted_avg_spend"]
)

print("\n=== BG/NBD (MCMC) + Gamma-Gamma (MAP) ===")
print(f"Training time: {mcmc_training_time:.2f} seconds")
print(f"Customers with predictions: {len(mcmc_clv)}")
print(f"Avg predicted 90-day CLV: ${mcmc_clv['predicted_clv'].mean():.2f}")
print("\nNote: MCMC provides uncertainty estimates (not shown here)")

## Model Comparison: Summary Table

In [ ]:
comparison = pd.DataFrame(
    [
        {
            "Model": "Historical Average",
            "Training Time": f"{historical_time:.3f}s",
            "Customers": len(historical_predictions),
            "Avg CLV": f"${historical_predictions['predicted_clv'].mean():.2f}",
            "Total Revenue": f"${historical_predictions['predicted_clv'].sum():,.2f}",
            "Personalized": "No",
        },
        {
            "Model": "BG/NBD + GG (MAP)",
            "Training Time": f"{map_training_time:.2f}s",
            "Customers": len(map_clv),
            "Avg CLV": f"${map_clv['predicted_clv'].mean():.2f}",
            "Total Revenue": f"${map_clv['predicted_clv'].sum():,.2f}",
            "Personalized": "Yes",
        },
        {
            "Model": "BG/NBD (MCMC) + GG (MAP)",
            "Training Time": f"{mcmc_training_time:.2f}s",
            "Customers": len(mcmc_clv),
            "Avg CLV": f"${mcmc_clv['predicted_clv'].mean():.2f}",
            "Total Revenue": f"${mcmc_clv['predicted_clv'].sum():,.2f}",
            "Personalized": "Yes",
        },
    ]
)

print("=== Model Comparison ===")
print(comparison.to_string(index=False))

## Prediction Distribution Comparison

Visualize how different models distribute CLV predictions.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Historical (all same value, so just show bar)
axes[0].hist(
    [historical_predictions["predicted_clv"].iloc[0]], bins=1, edgecolor="black"
)
axes[0].set_title("Historical Average\n(No Personalization)")
axes[0].set_xlabel("90-Day CLV ($)")
axes[0].set_ylabel("Frequency")
axes[0].axvline(
    historical_predictions["predicted_clv"].mean(),
    color="red",
    linestyle="--",
    linewidth=2,
)

# MAP
axes[1].hist(map_clv["predicted_clv"], bins=50, edgecolor="black", alpha=0.7)
axes[1].set_title("BG/NBD + Gamma-Gamma (MAP)")
axes[1].set_xlabel("90-Day CLV ($)")
axes[1].set_ylabel("Frequency")
axes[1].axvline(
    map_clv["predicted_clv"].mean(),
    color="red",
    linestyle="--",
    linewidth=2,
    label="Mean",
)
axes[1].legend()

# MCMC (subset)
axes[2].hist(
    mcmc_clv["predicted_clv"], bins=30, edgecolor="black", alpha=0.7, color="green"
)
axes[2].set_title("BG/NBD (MCMC) + GG (MAP)\n(200 customer sample)")
axes[2].set_xlabel("90-Day CLV ($)")
axes[2].set_ylabel("Frequency")
axes[2].axvline(
    mcmc_clv["predicted_clv"].mean(),
    color="red",
    linestyle="--",
    linewidth=2,
    label="Mean",
)
axes[2].legend()

plt.tight_layout()
plt.show()

print("📊 Personalized models (MAP/MCMC) show realistic CLV distribution")
print("   Historical average treats all customers identically")

## Test Under Different Scenarios

Compare how models perform with stable vs. high-churn customer bases.

In [ ]:
scenarios_to_test = [
    ("Stable Business", STABLE_BUSINESS_SCENARIO),
    ("High Churn", HIGH_CHURN_SCENARIO),
]

scenario_results = []

for scenario_name, scenario_config in scenarios_to_test:
    print(f"\nTesting: {scenario_name}")

    # Generate data
    test_customers = generate_customers(
        500, date(2024, 1, 1), date(2024, 12, 31), seed=99
    )
    test_transactions = generate_transactions(
        test_customers, date(2024, 1, 1), date(2024, 12, 31), scenario=scenario_config
    )

    test_data = prepare_clv_model_inputs(
        transactions=[asdict(t) for t in test_transactions],
        observation_start=datetime(2024, 1, 1),
        observation_end=datetime(2024, 12, 31, 23, 59, 59),
        customer_id_field="customer_id",
        timestamp_field="event_ts",
        monetary_field="unit_price",
    )

    # Train MAP model
    bg_nbd_test = BGNBDModelWrapper(BGNBDConfig(method="map"))
    bg_nbd_test.fit(test_data)

    test_predictions = bg_nbd_test.predict_purchases(test_data, time_periods=90.0)

    scenario_results.append(
        {
            "Scenario": scenario_name,
            "Customers": len(test_data),
            "Avg Frequency": test_data["frequency"].mean(),
            "Avg Predicted Purchases (90d)": test_predictions[
                "predicted_purchases"
            ].mean(),
        }
    )

    print(f"  Avg historical frequency: {test_data['frequency'].mean():.2f}")
    print(
        f"  Avg predicted purchases: {test_predictions['predicted_purchases'].mean():.2f}"
    )

scenario_df = pd.DataFrame(scenario_results)
print("\n=== Scenario Comparison ===")
print(scenario_df.to_string(index=False))

## Model Selection Guide

In [ ]:
guide = pd.DataFrame(
    [
        {
            "Method": "Historical Average",
            "Speed": "⚡⚡⚡ Instant",
            "Accuracy": "⭐ Low",
            "Personalization": "❌ No",
            "Best For": "Quick baseline, aggregate forecasts",
        },
        {
            "Method": "BG/NBD + GG (MAP)",
            "Speed": "⚡⚡ Fast (seconds)",
            "Accuracy": "⭐⭐⭐ High",
            "Personalization": "✅ Yes",
            "Best For": "Production deployments, real-time scoring",
        },
        {
            "Method": "BG/NBD + GG (MCMC)",
            "Speed": "⚡ Slow (minutes)",
            "Accuracy": "⭐⭐⭐⭐ Very High",
            "Personalization": "✅ Yes + Uncertainty",
            "Best For": "Research, model validation, small datasets",
        },
    ]
)

print("=== Model Selection Guide ===")
print(guide.to_string(index=False))

## Top Customer Comparison

See if different models identify the same high-value customers.

In [ ]:
# Get top 20 customers from each model
map_top20 = set(map_clv.nlargest(20, "predicted_clv")["customer_id"])
mcmc_top20 = set(mcmc_clv.nlargest(20, "predicted_clv")["customer_id"])

overlap = map_top20 & mcmc_top20

print("=== Top 20 Customer Overlap ===")
print(f"MAP top 20: {len(map_top20)} customers")
print(f"MCMC top 20: {len(mcmc_top20)} customers")
print(f"Overlap: {len(overlap)} customers ({len(overlap) / 20 * 100:.0f}%)")
print(
    f"\nConclusion: {'Strong agreement' if len(overlap) >= 15 else 'Models identify different high-value customers'}"
)

## Summary and Recommendations

### Key Findings

1. **Historical Average**: Fastest but least accurate; no personalization
2. **MAP**: Best balance of speed and accuracy for production use
3. **MCMC**: Most accurate with uncertainty estimates, but much slower

### When to Use Each Method

**Use Historical Average when:**
- You need a quick baseline for comparison
- Making aggregate forecasts (not individual predictions)
- Starting a new project and need quick insights

**Use MAP when:**
- Deploying to production (fast inference)
- Scoring customers in real-time or batch
- Dataset has 100+ customers with repeat purchases
- Speed matters more than marginal accuracy gains

**Use MCMC when:**
- Validating model assumptions
- Need uncertainty quantification
- Small datasets (<500 customers)
- Research or one-time analysis
- Computational resources available

### Performance Tips

- Start with MAP, upgrade to MCMC only if needed
- For MCMC, use 2-4 chains and 2000+ draws for production
- Test models on synthetic data before production deployment
- Monitor prediction stability across model updates